In [1]:
using NBInclude

@nbinclude("functions.ipynb")

All function imported


In [10]:
function get_test_loader(batch_size, shuffle::Bool)
    # The FashionMNIST test set is made up of 10k 28 by 28 greyscale images
    test_x, test_y = MNIST.testdata(Float32)
    test_x = 1 .- reshape(test_x, (784, :))
    return DataLoader((test_x, test_y), batchsize=batch_size, shuffle=shuffle)
end


# setup
# experiments with MNIST dataset
x_dim = 784
k = 20
epoch_to_load = 20 # Load the model 
tolerance = 1e-7; max_iter = 5000; out_toggle = 0
trials = 20
opt = Flux.Optimise.ADAM(.1);


# pick a image in MNIST to denoise
num = 6
batch_size = 64
shuffle_data = true
dataloader = get_test_loader(batch_size, shuffle_data)

(x_batch, y_batch) = first(dataloader)
i = 1
while y_batch[i] != num
    i += 1
end

x_true= x_batch[:,i];

epoch_to_load = 20 # Load the model 
encoder_μ_inco, encoder_logvar_inco,  decoder_inco, W3_inco = load_model_identity("trained_GNN/MNIST_identity_v2", epoch_to_load)
μ = encoder_μ_inco(x_true)
logvar_inco = encoder_logvar_inco(x_true)
# Apply reparameterisation trick to sample latent
z_inco = μ + randn(Float32, size(logvar_inco)) .* exp.(0.5f0 * logvar_inco)
z = z_inco

F = dct(diagm(ones(x_dim)),2);

W1_inco = Flux.params(decoder_inco)[1]
W2_inco = Flux.params(decoder_inco)[2]


_, _, decoder = load_model("trained_GNN/MNIST", 20)
A = Flux.params(decoder)[3]

W1_co = Flux.params(decoder)[1]
W2_co = Flux.params(decoder)[2]

β_list, α_list = get_β_α(F, A, decoder_last, 11);
m_list = 20:20:440


┌ Warning: MNIST.testdata() is deprecated, use `MNIST(split=:test)[:]` instead.
└ @ MLDatasets C:\Users\Babhru\.julia\packages\MLDatasets\Xb4Lh\src\datasets\vision\mnist.jl:195


Loading model...Done
Loading model...

Done


20:20:440

In [29]:
_,s,_, =svd(W3_inco);
s

500-element Vector{Float64}:
 1.0059194266636657
 1.0035087786247583
 1.0034389191296542
 1.003373420517567
 1.0033208066565054
 1.0032690291877637
 1.003263575726526
 1.0032392488281083
 1.0031834447901518
 1.0031702933993867
 ⋮
 0.9948865887716473
 0.9941517303695564
 0.9940395536667713
 0.993781467662166
 0.9933933832223779
 0.9931360031363893
 0.9930274063506201
 0.9925571425106124
 0.9922919804740524

In [12]:
phase_matrix_recovery = zeros(trials, length(m_list), length(β_list))
phase_matrix_recon = zeros(trials, length(m_list), length(β_list))

Threads.@threads for i in 1:length(m_list)
    m = m_list[i]
    recovery_error_matrix = zeros(trials)
    recon_error_matrix = zeros(trials)
    for β in β_list
        recovery_error_list = []
        recon_error_list = []
        G = Chain(
            Dense(20, 500, relu, bias = false; init =(out,in) ->  Flux.params(decoder_inco)[1]),
            Dense(500, 500, relu, bias = false; init =(out,in) -> Flux.params(decoder_inco)[2]),
            Dense(500, 784, identity, bias = false; init =(out,in) -> β *A + (1-β) * W3_inco),
        )      #setup the generative network 

        for trials in 1:trials
            F_sub = sample_fourier(m, x_dim)  # subsampling DCT for measurement matrix
            F_sub = F_sub * sqrt(x_dim)/sqrt(m) #normalization          
            y = F_sub * G(z) # measurement vector using subsampled DCT matrix

            z_est = randn(k) #initialization for algorithm
            z_est = estimated_code(opt, G, y, F_sub, z_est; max_iter, tolerance, out_toggle)  # run a optimizer to solve the least squares problem

            push!(recovery_error_list, relative_error(z, z_est))    
            push!(recon_error_list, relative_error(G(z), G(z_est)))
        end

        recovery_error_matrix =  hcat(recovery_error_matrix, recovery_error_list)
        recon_error_matrix = hcat(recon_error_matrix, recon_error_list)
    end
    phase_matrix_recovery[:, i, :] = recovery_error_matrix[:, 2:end]
    phase_matrix_recon[:, i, :] = recon_error_matrix[:, 2:end]
    println("Done with m = $m")
println("")
end

Done with m = 260

Done with m = 200

Done with m = 140

Done with m = 420

Done with m = 320

Done with m = 380

Done with m = 80

Done with m = 280

Done with m = 160

Done with m = 220

Done with m = 340

Done with m = 400

Done with m = 440

Done with m = 100

Done with m = 20

Done with m = 300

Done with m = 240

Done with m = 180

Done with m = 360

Done with m = 120



Done with m = 40



Done with m = 60



In [8]:
save("saved_data/Phase_plot_full_MNIST_v2.jld", "phase_matrix_recovery", phase_matrix_recovery, "phase_matrix_recon", phase_matrix_recon, "α_list", α_list, "m_list", m_list, "trials", trials)

In [13]:
M = zeros(size(phase_matrix_recovery[1,:,:]))
for trial in 1:trials
    M += (phase_matrix_recovery[trial,:,:] .< 1e-2)
end
M

22×11 Matrix{Float64}:
  0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0
 17.0  18.0  16.0  19.0  17.0  19.0  18.0  19.0  18.0  13.0   0.0
 20.0  18.0  20.0  19.0  20.0  20.0  20.0  19.0  17.0  14.0   0.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  16.0   1.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  16.0   4.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  18.0  18.0   5.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  19.0  18.0   5.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  18.0   6.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0   7.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  19.0   9.0
  ⋮                             ⋮                             ⋮
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  19.0  13.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  15.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  11.0
 20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  20.0  

In [16]:
M = zeros(size(phase_matrix_recovery[1,:,:]))
for trial in 1:trials
    M += (phase_matrix_recovery[trial,:,:] .< 1e-5)
end
p1 = plot(colorview(Gray, reverse(M/trials, dims = 1)), bg =:black, title = "recovery of code z₀", framestyle = :box)
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 2:5:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 2:5:length(m_list)])

# m_pred = 600*α_list.^10
# ff(x) = -21/420 * x +(1 + 21*440/420)
# plot!(1:length(α_list), ff.(m_pred), label =:false, linewidth = 2 )

# m_pred = 150*α_list.^3
# plot!(1:length(α_list), ff.(m_pred), label =:false, linewidth = 2 )


M = zeros(size(phase_matrix_recon[1,:,:]))
for trial in 1:trials
    M += (phase_matrix_recovery[trial,:,:] .< 1e-5)
end

p2 = plot(colorview(Gray, reverse(M/trials, dims = 1)), bg =:black, title = "recovery of signal G(z₀)",framestyle = :box)
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 2:5:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 2:5:length(m_list)])
# plot!(1:length(α_list), ff.(m_pred), label =:false, linewidth = 2 )

plot(p1, p2, layout = 2, xlabel = "incoherence upper bound", ylabel = "number of measurements")
savefig("figures/tests/Full_MNIST_inner_high_last_low.pdf")



In [95]:


M = sum(phase_matrix_recovery,dims =1)[1,:,:]/20; M[M .>1] .= 1
p1 = plot(colorview(Gray, reverse(M,dims=1)), framestyle = :box, bg =:black, title = "recovery of code z₀")
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 2:5:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 2:5:length(m_list)])

M = sum(phase_matrix_recon,dims =1)[1,:,:]/20; M[M .>1] .= 1
p2 = plot(colorview(Gray, reverse(M,dims=1)), framestyle = :box, bg =:black, title = "recovery of signal G(z₀)")
xticks = 1:5:length(α_list)
xticks!(xticks, ["$(round.(α_list, digits = 2)[i])" for i in 1:5:length(α_list)])
yticks = 2:5:length(m_list)
yticks!(yticks, ["$(m_list[length(m_list) + 1 - i])" for i in 2:5:length(m_list)])

plot(p1, p2, layout = 2,  bg =:white)
savefig("figures/Phase_plot_Full_MNIST_error.pdf")

